In [1]:
from dotenv import load_dotenv
import os
from langchain.llms import OpenAI
from langchain import HuggingFaceHub

import warnings
warnings.filterwarnings('ignore')


In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN=os.getenv('HUGGINGFACEHUB_API_TOKEN')

# Getting Started with LangChain

### OpenAI

In [3]:
llm=OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.7)

In [4]:
text = "Who is Eren Yeager?"
print(llm.predict(text))



Eren Yeager is the main protagonist of the popular Japanese manga and anime series, Attack on Titan. He is a young man who lives in a world where humanity is constantly under threat from giant humanoid creatures called Titans. Eren joins the military along with his childhood friends, Mikasa Ackerman and Armin Arlert, to fight against the Titans and seek revenge for the death of his mother. He possesses the ability to transform into a Titan himself, making him a key player in the ongoing war against the Titans. Eren is determined, hot-headed, and driven by a strong desire to protect his loved ones and eradicate the Titans.


### Google FLAN

In [5]:
llm_hf_flan=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})

In [6]:
output=llm_hf_flan.predict("Who is Eren Yeager?")
print(output)

physicist


In [7]:
output=llm_hf_flan.predict("Write a poem about leafs")
print(output)

leafs are a beautiful thing they are a beautiful thing they are a beautiful thing


### Mixtral-8x7B

In [8]:
llm_hf_mixtral=HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",model_kwargs={"temperature":0.7,"max_length":64})

In [9]:
output=llm_hf_mixtral.predict("Who is Eren Yeager?")
print(output)

Who is Eren Yeager?

Eren Yeager is the main protagonist of the anime and manga series Attack on Titan. He is a young man who has vowed to exterminate the Titans, giant humanoid creatures that have been terrorizing humanity for centuries. Eren is driven by a deep hatred for the Titans, as they were responsible for the death of his mother when he was a child.

Eren is a member of the Survey Corps, an elite military unit that


# Prompt Templates And LLMChain

In [10]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['anime_name'],
template="Give the plot summary of the anime- {anime_name}")

prompt_template.format(anime_name="Attack on Titan")

'Give the plot summary of the anime- Attack on Titan'

In [11]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm_hf_mixtral,prompt=prompt_template)
print(chain.run("Attack on Titan"))


Give the plot summary of the anime- Attack on Titan.

Attack on Titan is a dark fantasy anime series that follows the story of Eren Yeager, his adopted sister Mikasa Ackerman, and their friend Armin Arlert. The story is set in a world where humanity lives inside three massive walls that protect them from the Titans, giant humanoid creatures that eat humans without reason.

The series begins with the sudden appearance of a Colossal Titan, who breaches the outer wall of the city


# Combining Multiple Chains with simple Sequential Chain

In [12]:
protagonist_template=PromptTemplate(input_variables=['anime_name'],
                                    template="Tell me the name of the protagonist of {anime_name}")
protagonist_chain=LLMChain(llm=llm_hf_mixtral,prompt=protagonist_template)

description_template=PromptTemplate(input_variables=['description'],
                                    template="Give a description of - {description}")
description_chain=LLMChain(llm=llm_hf_mixtral,prompt=description_template)

In [13]:
from langchain.chains import SimpleSequentialChain

chain=SimpleSequentialChain(chains=[protagonist_chain,description_chain])
print(chain.run("Attack on Titan"))

Give a description of - Tell me the name of the protagonist of Attack on Titan.

Eren Jaeger

Attack on Titan is a Japanese manga series written and illustrated by Hajime Isayama. The story is set in a world where humanity lives in cities surrounded by enormous walls that protect them from gigantic humanoid creatures known as Titans. The protagonist, Eren Jaeger, along with his friends Mikasa Ackerman and Armin Arlert, join the military to fight against the Titans after their home town is destroyed and his mother is eaten by a Titan. Eren has a strong desire to defeat the Titans and uncover the truth behind their existence. He also discovers that he has the ability to transform into a Titan, which becomes a crucial plot point in the series.


# Sequential Chain

In [14]:
protagonist_template=PromptTemplate(input_variables=['anime_name'],
                                    template="Protagonist of {anime_name} is")

protagonist_chain=LLMChain(llm=llm_hf_mixtral,prompt=protagonist_template,output_key="protagonist")

In [15]:
description_template=PromptTemplate(input_variables=['protagonist'],
                                    template="describe {protagonist}")

description_chain=LLMChain(llm=llm_hf_mixtral,prompt=description_template,output_key="description")

In [16]:
from langchain.chains import SequentialChain

chain=SequentialChain(chains=[protagonist_chain,description_chain],
                      input_variables=['anime_name'],
                      output_variables=['protagonist',"description"])

In [17]:
chain({'anime_name':"Vinland Saga"})

{'anime_name': 'Vinland Saga',
 'protagonist': 'Protagonist of Vinland Saga is Thorfinn, a young Viking warrior who is the son of Thors, a legendary warrior who was once known as the Troll of Jom. Thorfinn is a skilled fighter and is determined to avenge his father’s death by killing Askeladd, the leader of the band of Vikings who killed Thors.\n\nThorfinn is a complex character who is driven by a deep sense of loyalty and a desire for justice. He is also haunted by the',
 'description': 'describe Protagonist of Vinland Saga is Thorfinn, a young Viking warrior who is the son of Thors, a legendary warrior who was once known as the Troll of Jom. Thorfinn is a skilled fighter and is determined to avenge his father’s death by killing Askeladd, the leader of the band of Vikings who killed Thors.\n\nThorfinn is a complex character who is driven by a deep sense of loyalty and a desire for justice. He is also haunted by the memory of his father and the guilt he feels for not being able to prot

# Chatmodels

### Hugging Face H4 Zephyr-7B

In [18]:
from langchain_community.llms import HuggingFaceHub

os.environ["HF_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

chatllm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.6,
        "repetition_penalty": 1.03,
    },
)

In [19]:
from langchain.schema import HumanMessage, SystemMessage
from langchain_community.chat_models.huggingface import ChatHuggingFace

messages = [SystemMessage(content="You're an acclaimed lyricist"),
            HumanMessage(content="Write a song for a fantasy thriller anime"),
]

chat_model = ChatHuggingFace(llm=chatllm)

WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [20]:
chat_model._to_chat_prompt(messages)

"<|system|>\nYou're an acclaimed lyricist</s>\n<|user|>\nWrite a song for a fantasy thriller anime</s>\n<|assistant|>\n"

In [21]:
res = chat_model.invoke(messages)
print(res.content)

<|system|>
You're an acclaimed lyricist</s>
<|user|>
Write a song for a fantasy thriller anime</s>
<|assistant|>
Verse 1:
In a world of magic and mystery
Where darkness lurks and danger's near
A hero rises, with heart and might
To face the shadows and conquer the night

Chorus:
Fantasy thriller, the battle begins
Adventure awaits, the fate of the world spins
With courage and strength, we'll face the unknown
Fantasy thriller, the journey's been shown

Ver


# Prompt Template + LLM +Output Parsers

In [22]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [23]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [24]:
template="Your are a helpful assistant. When the user gives a word input, you should generate 5 synonyms of that word in a comma seperated list"
human_template="{text}"

chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [25]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [26]:
print(chain.invoke({"text":"story"}))

['System: Your are a helpful assistant. When the user gives a word input', ' you should generate 5 synonyms of that word in a comma seperated list\nHuman: story\nAssistant: narrative', ' tale', ' account', ' history', ' anecdote']
